Extended model of the implementation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random

In [ ]:
class DSO:
	def __init__(self, totalHours, marginalCost, unitSalePrice, maxPriceReference):
		self.totalHours = totalHours
		self.numberOfAggregators = 0
		self.marginalCost = marginalCost
		self.unitSalePrice = unitSalePrice
		self.maxPriceReference = maxPriceReference
		self.aggregators = []
		self.maxUtility = 0
		self.m = 0

	def addAggregator(self, aggregator):
		self.aggregators.append(aggregator)
		self.numberOfAggregators = self.numberOfAggregators + 1

	def getAggregators(self):
		return self.aggregators

	def Sfunction(self, aggregator, tIndex):
		return aggregator.maximumDemand[tIndex] * aggregator.maxPriceReference[tIndex] * (1 - np.exp(-aggregator.preferenceSatisfaction[tIndex] * (aggregator.bestLoadResponse[tIndex] / aggregator.nominalDemand[tIndex])))
	
	def CostFunction(self, aggregator, tIndex):
		return self.marginalCost[tIndex]
	
	def getLeftLimit(self):
		leftLimit = [0 for i in range(self.totalHours)]
		for i in range(self.totalHours):
			mx = 0
			for j in range(self.numberOfAggregators):
				agg = self.aggregators[j]
				mx = max(mx, agg.preferenceSatisfaction[i] * agg.maxPriceReference[i] * np.exp(-agg.preferenceSatisfaction[i] * (agg.maximumDemand[i] / agg.nominalDemand[i])))
			leftLimit[i] = mx
		return leftLimit
	
	def getRightLimit(self):
		rightLimit = [0 for i in range(self.totalHours)]
		for i in range(self.totalHours):
			mx = 0
			for j in range(self.numberOfAggregators):
				agg = self.aggregators[j]
				mx = max(mx, agg.preferenceSatisfaction[i] * agg.maxPriceReference[i] * np.exp(-agg.preferenceSatisfaction[i] * (agg.minimumDemand[i] / agg.nominalDemand[i])))
			rightLimit[i] = mx
		return rightLimit
	
	def getUnitSalePriceRange(self):
		minPrice = self.marginalCost
		leftLimit = self.getLeftLimit()
		minPrice = np.minimum(minPrice, leftLimit)

		maxPrice = self.maxPriceReference
		rightLimit = self.getRightLimit()
		maxPrice = np.maximum(maxPrice, rightLimit)

		return minPrice, maxPrice
	
	def setUnitSalePrice(self, unitSalePrice, tIndex):
		self.unitSalePrice[tIndex] = unitSalePrice

	def generateM(self):
		m = 0
		mx = 0
		for i in range(self.totalHours):
			sum = 0
			for j in range(self.numberOfAggregators):
				agg = self.aggregators[j]
				sum = sum + agg.bestLoadResponse[i]
			mx = max(mx, sum)
		m = random.uniform(mx, mx*10)
		self.m = m

	def utilityFunction(self, m, theta, omega):
		self.generateM()
		print('m value:', self.m)

		mx = -1000000000000
		for j in range(self.totalHours):
			sum = 0
			for i in range(self.numberOfAggregators):
				agg = self.aggregators[i]
				sum = sum + self.unitSalePrice[j] * agg.bestLoadResponse[j] - self.CostFunction(agg, j) * agg.bestLoadResponse[j] + omega * self.Sfunction(agg, j)
			sum = sum - theta*m
			mx = max(mx, sum)
		return mx
	
	def partialUtilityFunction(self, m, theta, omega, tIndex):
		sum = 0
		for i in range(self.numberOfAggregators):
			agg = self.aggregators[i]
			sum = sum + self.unitSalePrice[tIndex] * agg.bestLoadResponse[tIndex] - self.CostFunction(agg, tIndex) * agg.bestLoadResponse[tIndex] + omega * self.Sfunction(agg, tIndex)
		return sum

	def getPAR(self):
		sum = 0
		for i in range(self.totalHours):
			for j in range(self.numberOfAggregators):
				sum = sum + self.aggregators[j].bestLoadResponse[i]
		return (self.m * self.totalHours) / (sum)

class Aggregator:
	def __init__(self, totalHours, nominalDemand, minimumDemand, maximumDemand, maxPriceReference, preferenceSatisfaction):
		self.totalHours = totalHours
		self.nominalDemand = nominalDemand
		self.minimumDemand = minimumDemand
		self.maximumDemand = maximumDemand
		self.maxPriceReference = maxPriceReference
		self.preferenceSatisfaction = preferenceSatisfaction
		self.currentUtility = 0
		self.bestLoadResponse = [0 for i in range(totalHours)]

	def utilityFunction(self, DSO):
		sum = 0
		for i in range(DSO.totalHours):
			sum = sum + DSO.Sfunction(self, i) - (DSO.unitSalePrice[i] * self.bestLoadResponse[i])
		return sum

	def optimalDR(self, DSO, j):
			return ((self.nominalDemand[j] / self.preferenceSatisfaction[j]) * (np.log((self.preferenceSatisfaction[j] * self.maxPriceReference[j]) / DSO.unitSalePrice[j])))

	def checkConstraints(self, tIndex):
		flag = True
		if self.bestLoadResponse[tIndex] < self.minimumDemand[tIndex]:
			flag = False
		if self.bestLoadResponse[tIndex] > self.maximumDemand[tIndex]:
			flag = False
		return flag


In [ ]:
datasetPath = '../dataset/'

In [ ]:
numOfLA = 3
numOfHours = 50

In [ ]:
nominalDemands = []
for i in range (0, 3):
	letter = chr(ord('A') + i)
	fileName = 'Home' + letter + '_hourly.csv'
	df = pd.read_csv(datasetPath + fileName)
	df = df[0:numOfHours]
	# Power is in kW
	nominalDemands.append(df['Power'].tolist())

In [ ]:
LAs = []
for i in range(numOfLA):
	nominalDemand = nominalDemands[i]
	maxNominalDemand = max(nominalDemand)
	minimumDemand = np.random.uniform(0.0001, nominalDemand, numOfHours)
	maximumDemand = np.random.uniform(nominalDemand, maxNominalDemand+1, numOfHours)
	preferenceSatisfaction = np.random.uniform(0.1, 15, numOfHours)
	maxPriceReference = np.random.randint(1000, 10000000, numOfHours)
	LAs.append(Aggregator(numOfHours, nominalDemand, minimumDemand, maximumDemand, maxPriceReference, preferenceSatisfaction))


In [ ]:
marginalCost = np.random.randint(1, 7, numOfHours) # cent/kW
unitSalePrice = np.random.randint(1, 7, numOfHours) # cent/kWh
dso = DSO(numOfHours, marginalCost, unitSalePrice, maxPriceReference)

In [ ]:
# append all the LAs to the DSO
for i in range(numOfLA):
	dso.addAggregator(LAs[i])

In [ ]:
omega = 5
theta = 0.01
minP, maxP = dso.getUnitSalePriceRange()
flag1 = True
while flag1:
    noValues = [True for i in range(numOfHours)]
    for i in range(numOfHours):
        flag2 = True
        temp = 0
        while flag2:
            price = minP[i]
            flag3 = [False for i in range(numOfLA)]
            flag4 = [False for i in range(numOfLA)]
            flag5 = [False for i in range(numOfLA)]
            while price <= maxP[i]:
                # print("price: ", price)
                dso.setUnitSalePrice(price, i)
                flag = [False for i in range(numOfLA)]
                for j in range(numOfLA):
                    actualLoad = LAs[j].optimalDR(dso,i)
                    LAs[j].bestLoadResponse[i] = actualLoad
                    flag[j] = LAs[j].checkConstraints(i)
                if flag.count(True) == 3:
                    flag2 = False
                    temp = price
                    noValues[i] = False
                    break
                #if any two values of flag are true and the rest are false
                t1=0
                t2=0
                t3=0
                if flag.count(True) == 2 and flag.count(False) == 1:
                    t1 = price
                    flag3 = flag
                if flag.count(True) == 1 and flag.count(False) == 2:
                    t2 = price
                    flag4 = flag
                if flag.count(True) == 0 and flag.count(False) == 3:
                    t3 = price
                    flag5 = flag
                price += 100
            if t1!=0:
                index = flag3.index(False)
                difference = dso.unitSalePrice[i] - LAs[index].maxPriceReference[i]
                LAs[index].maxPriceReference[i] = dso.unitSalePrice[i] + 100
                LAs[index].preferenceSatisfaction[i] = LAs[index].preferenceSatisfaction[i] * difference/dso.unitSalePrice[i]
                if (LAs[0].checkConstraints(i) and LAs[1].checkConstraints(i) and LAs[2].checkConstraints(i)):
                    flag2 = False
                    temp = price
                    noValues[i] = False
                    break
            elif t2!=0:
                #do the same as above but for both the LAs with false values
                index1 = flag4.index(False)
                index2 = flag4.index(False, index1+1)
                difference1 = dso.unitSalePrice[i] - LAs[index1].maxPriceReference[i]
                difference2 = dso.unitSalePrice[i] - LAs[index2].maxPriceReference[i]
                LAs[index1].maxPriceReference[i] = dso.unitSalePrice[i] + 100
                LAs[index1].preferenceSatisfaction[i] = LAs[index1].preferenceSatisfaction[i] * difference1/dso.unitSalePrice[i]
                LAs[index2].maxPriceReference[i] = dso.unitSalePrice[i] + 100
                LAs[index2].preferenceSatisfaction[i] = LAs[index2].preferenceSatisfaction[i] * difference2/dso.unitSalePrice[i]
                if (LAs[0].checkConstraints(i) and LAs[1].checkConstraints(i) and LAs[2].checkConstraints(i)):
                    flag2 = False
                    temp = price
                    noValues[i] = False
                    break
            elif t3!=0:
                #do the same as above but for all the LAs
                for j in range(numOfLA):
                    difference = dso.unitSalePrice[i] - LAs[j].maxPriceReference[i]
                    LAs[j].maxPriceReference[i] = dso.unitSalePrice[i] + 100
                    LAs[j].preferenceSatisfaction[i] = LAs[j].preferenceSatisfaction[i] * difference/dso.unitSalePrice[i]
                if (LAs[0].checkConstraints(i) and LAs[1].checkConstraints(i) and LAs[2].checkConstraints(i)):
                    flag2 = False
                    temp = price
                    noValues[i] = False
                    break
        dso.setUnitSalePrice(temp, i)
    if all(item == False for item in noValues):
        flag1 = False


In [ ]:
# # save the variables for each LA
# for i in range(numOfLA):
# 	letter = chr(ord('A') + i)
# 	fileName = 'Home' + letter + '_hourly.csv'
# 	df = pd.read_csv(datasetPath + fileName)
# 	df = df[0:numOfHours]
# 	df['nominalDemand'] = LAs[i].nominalDemand
# 	df['minimumDemand'] = LAs[i].minimumDemand
# 	df['maximumDemand'] = LAs[i].maximumDemand
# 	df['preferenceSatisfaction'] = LAs[i].preferenceSatisfaction
# 	df['bestLoadResponse'] = LAs[i].bestLoadResponse
# 	df.to_csv(datasetPath + 'Home' + letter + '_hourly_extended_results.csv', index=False)

In [ ]:
# # save the variables for the DSO
# df = pd.DataFrame()
# df['marginalCost'] = dso.marginalCost
# df['unitSalePrice'] = dso.unitSalePrice
# df['PAR'] = dso.getPAR()
# df.to_csv(datasetPath + 'DSO_hourly_extended_results.csv', index=False)

In [ ]:
# print Unit Sale Price
print("Unit Sale Price: ", dso.unitSalePrice)

In [ ]:
# DSO gives the price for each hour & each LA calculates the optimal DR
for i in range(numOfLA):
	# la = LAs[i]
	for j in range(numOfHours):
		# la.optimalDR(dso, j)
		LAs[i].optimalDR(dso, j)

In [ ]:
# print best load response for each LA
for i in range(numOfLA): # kWh
	print("LA", i)
	print("best load response: ", dso.aggregators[i].bestLoadResponse)

In [ ]:
# DSO takes back the load response from each aggregator
m = np.random.randint(1, 100)
omega = 5
theta = 0.01

dso.maxUtility = dso.utilityFunction(m, theta, omega)
dso.maxUtility

In [ ]:
# DSO takes back the load response from each aggregator
m = np.random.randint(1, 100)
omega = 6
theta = 0.02

dso.maxUtility = dso.utilityFunction(m, theta, omega)
dso.maxUtility

In [ ]:
# DSO takes back the load response from each aggregator
m = np.random.randint(1, 100)
omega = 2
theta = 0.1

dso.maxUtility = dso.utilityFunction(m, theta, omega)
dso.maxUtility

In [ ]:
# print utility for each house
for i in range(numOfLA):
	print("LA", i)
	ut = LAs[i].utilityFunction(dso)
	print("utility: ", ut)

In [ ]:
dso.getPAR()

In [ ]:
# plot nominal demand of each LA
plt.figure(figsize=(20,8), dpi=300)
plt.xlabel('Hour', fontsize=20)
plt.ylabel('kWh', fontsize=20)
plt.xticks(np.arange(0, numOfHours, 5), fontsize=15)
plt.yticks(np.arange(0, 10, 2), fontsize=15)
for i in range(numOfLA):
	plt.plot(LAs[i].nominalDemand)
plt.legend([chr(ord('A') + i) for i in range(numOfLA)], fontsize=30)
plt.show()

In [ ]:
# plot user pref of each LA
plt.figure(figsize=(50, 12), dpi=300)
plt.xticks(np.arange(0, numOfHours, 5), fontsize=40)
plt.yticks(fontsize=40)
plt.xlabel('Hour', fontsize=40)
plt.ylabel('User Preference', fontsize=40)
for i in range(numOfLA):
	plt.plot(LAs[i].preferenceSatisfaction)
plt.legend([chr(ord('A') + i) for i in range(numOfLA)], fontsize=40)
plt.show()

In [ ]:
# plot marginal cost
plt.plot(dso.marginalCost)
plt.xticks(np.arange(0, numOfHours, 5), fontsize=8)
plt.yticks(np.arange(0, 7, 1), fontsize=8)
plt.xlabel('Hour', fontsize=8)
plt.ylabel('Marginal Cost', fontsize=8)
plt.show()

In [ ]:
# plot bestLoadResponse of each LA
plt.figure(figsize=(50, 12), dpi=300)
plt.xticks(np.arange(0, numOfHours, 5), fontsize=40)
plt.yticks(np.arange(0, 15, 2), fontsize=40)
plt.xlabel('Hour', fontsize=40)
plt.ylabel('kWh', fontsize=40)
for i in range(numOfLA):
	plt.plot(LAs[i].bestLoadResponse)
plt.legend([chr(ord('A') + i) for i in range(numOfLA)], fontsize=40)
plt.show()